# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118703e+02     1.513966e+00
 * time: 0.07802009582519531
     1     1.112883e+01     8.989693e-01
 * time: 1.444167137145996
     2    -1.177817e+01     9.145720e-01
 * time: 1.5481171607971191
     3    -3.418889e+01     7.211346e-01
 * time: 1.6838390827178955
     4    -4.780127e+01     5.590095e-01
 * time: 1.8844561576843262
     5    -5.691863e+01     2.281750e-01
 * time: 2.008315086364746
     6    -5.981740e+01     1.209034e-01
 * time: 2.1045539379119873
     7    -6.091784e+01     5.037106e-02
 * time: 2.201064109802246
     8    -6.133997e+01     8.250766e-02
 * time: 2.3280861377716064
     9    -6.166352e+01     3.604097e-02
 * time: 2.424323081970215
    10    -6.187485e+01     2.468195e-02
 * time: 2.519350051879883
    11    -6.202231e+01     1.710251e-02
 * time: 2.614192008972168
    12    -6.207652e+01     1.836437e-02
 * time: 2.7123570442199707
    13    -6.212967e+01     1.819060e-02
 * time: 2.812631130218506

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671064
    AtomicLocal         -18.8557645
    AtomicNonlocal      14.8522621
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485366 
    Xc                  -19.3336818

    total               -62.261666457704
